<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool, OpenAIChatCompletionsModel, set_default_openai_client, set_tracing_export_api_key
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account
from openai import AsyncOpenAI, AsyncAzureOpenAI

load_dotenv(override=True)

openai = os.getenv("OPENAI_API_KEY")
openai_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="gtp-4o-mini"
)

gemini_client = AsyncOpenAI(base_url=os.getenv('GEMINI_API_BASE'), api_key=os.getenv('GEMINI_API_KEY'))
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.5-flash",openai_client=gemini_client)

set_default_openai_client(gemini_client);
set_tracing_export_api_key(openai);

### Let's start by gathering the MCP params for our trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "python3", "args": ["-m", "mcp_server_fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [6]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model=gemini_model,
        mcp_servers=mcp_servers,
    )
    return researcher

In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [8]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



Here's a summary of the latest news on Amazon, based on recent web searches:

Amazon continues to be active across its various sectors:

*   **Official News & Principles:** Amazon's official news channels (aboutamazon.com/news) highlight the company's guiding principles: customer obsession, passion for invention, commitment to operational excellence, and long-term thinking. These channels provide updates on Amazon's impact in communities, environmental commitments, innovation, and stories about sellers.
*   **AWS Innovations:** Amazon Web Services (AWS) is constantly expanding, with daily announcements and launches of new cloud innovations. For instance, there's recent news about new OpenAI open-weight models available in Amazon Bedrock, which are highlighted for their significant price-performance improvements compared to other models.
*   **Amazon Games:** Amazon Games regularly releases news regarding game updates, new game releases, business partnerships, and insights into their work culture.
*   **Community Impact:** Amazon's social media often features community-focused stories, such as surprising a 16-year-old who delivered supplies to foster kids by matching his donation.
*   **Financial News:** For stock-related information and financial news, resources like Yahoo Finance and CNBC provide real-time quotes, historical data, and financial reporting for Amazon.com, Inc. (AMZN).
*   **Seller Updates:** For third-party sellers, dedicated news channels (like myamazonguy.com) provide updates on Amazon Seller News, offering insights to help sellers enhance visibility, improve sales, and stay competitive in the evolving marketplace.

### Look at the trace

https://platform.openai.com/traces

In [9]:
adrian_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Adrian").reset(adrian_initial_strategy)

display(Markdown(await read_accounts_resource("Adrian")))
display(Markdown(await read_strategy_resource("Adrian")))

{"name": "adrian", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-25 14:20:00", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [10]:
agent_name = "Adrian"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [11]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Adrian and your account is under your name, Adrian.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "adrian", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-08-25 14:20:00", 10000.0], ["2025-08-25 14:20:02", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so

### And to run our Trader

In [12]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model=gemini_model,
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

I have purchased 200 shares of Intel (INTC) for Adrian's account at a price of 24.8496 per share, totaling 4969.92. This decision was based on news that SoftBank has injected $2 billion into Intel and the US is also building a stake, which is expected to cause a short-term upward price movement. My account balance is now 5030.08, and the total portfolio value is 9990.08. I will monitor the stock for a quick profit opportunity, consistent with my day trading strategy.

### Then go and look at the trace

http://platform.openai.com/traces


In [13]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "adrian", "balance": 5030.08, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"INTC": 200}, "transactions": [{"symbol": "INTC", "quantity": 200, "price": 24.849600000000002, "timestamp": "2025-08-25 14:20:27", "rationale": "Intel (INTC) stock pumped 5% due to a $2 billion injection by SoftBank, with the US also set to build a stake. This positive news is likely to cause short-term upward price movement, presenting a good day trading opportunity."}], "portfolio_value_time_series": [["2025-08-25 14:20:00", 10000.0], ["2025-08-25 14:20:02", 10000.0], ["2025-08-25 14:20:27", 9990.08], ["2025-08-25 14:20:30", 9990.08]], "total_portfolio_value": 9990.08, "total_profit_loss": -9.920000000000073}'

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [14]:
from traders import Trader


In [15]:
trader = Trader("Adrian", model_name="gemini-2.5-flash")

In [16]:
await trader.run()

In [17]:
await read_accounts_resource("Adrian")

'{"name": "adrian", "balance": 9980.16, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [{"symbol": "INTC", "quantity": 200, "price": 24.849600000000002, "timestamp": "2025-08-25 14:20:27", "rationale": "Intel (INTC) stock pumped 5% due to a $2 billion injection by SoftBank, with the US also set to build a stake. This positive news is likely to cause short-term upward price movement, presenting a good day trading opportunity."}, {"symbol": "INTC", "quantity": -200, "price": 24.7504, "timestamp": "2025-08-25 14:21:39", "rationale": "The initial catalyst for buying Intel (INTC) has not resulted in significant short-term upward price movement. As a day trader, I am cutting losses to adhere to my aggressive strategy of not holding underperforming assets when new, immediate opportunities are not present."}], "portfolio_value_time_series": [["2025-08-25 14:20:00", 10000.0], ["2025-08-25 14:20:02"

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [18]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("Adrian")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 16 tools
